In [2]:
%pip install autots
%pip install statsmodels


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# # also: _hourly, _daily, _weekly, or _yearly
# from autots.datasets import load_monthly

# df_long = load_monthly(long=True)

# from autots import AutoTS

# model = AutoTS(
#     forecast_length=3,
#     frequency='infer',
#     ensemble='simple',
#     max_generations=5,
#     num_validations=2,
# )
# model = model.fit(df_long, date_col='datetime', value_col='value', id_col='series_id')

# # Print the description of the best model
# print(model)

In [10]:
# help(model)

Help on AutoTS in module autots.evaluator.auto_ts object:

class AutoTS(builtins.object)
 |  AutoTS(forecast_length: int = 14, frequency: str = 'infer', prediction_interval: float = 0.9, max_generations: int = 10, no_negatives: bool = False, constraint: float = None, ensemble: str = None, initial_template: str = 'General+Random', random_seed: int = 2022, holiday_country: str = 'US', subset: int = None, aggfunc: str = 'first', na_tolerance: float = 1, metric_weighting: dict = {'smape_weighting': 5, 'mae_weighting': 2, 'rmse_weighting': 2, 'made_weighting': 0.5, 'mage_weighting': 0, 'mle_weighting': 0, 'imle_weighting': 0, 'spl_weighting': 3, 'containment_weighting': 0, 'contour_weighting': 1, 'runtime_weighting': 0.05, 'oda_weighting': 0.001}, drop_most_recent: int = 0, drop_data_older_than_periods: int = None, model_list: str = 'default', transformer_list: dict = 'auto', transformer_max_depth: int = 6, models_mode: str = 'random', num_validations: int = 'auto', models_to_validate: floa

In [1]:
from autots import AutoTS
from autots.datasets import load_hourly

df_wide = load_hourly(long=False)

# here we care most about traffic volume, all other series assumed to be weight of 1
weights_hourly = {'traffic_volume': 20}

df_wide

,clouds_all,rain_1h,snow_1h,temp,traffic_volume,weather_main
datetime,,,,,,
2016-10-01 00:00:00,20.0,0.0,0.0,286.22,1195.0,Clouds
2016-10-01 01:00:00,20.0,0.0,0.0,285.60,776.0,Clouds
2016-10-01 02:00:00,56.0,0.0,0.0,285.49,666.0,Clouds
2016-10-01 03:00:00,56.0,0.0,0.0,284.59,448.0,Clouds
2016-10-01 04:00:00,56.0,0.0,0.0,284.26,512.0,Clouds
...,...,...,...,...,...,...
2018-09-30 19:00:00,75.0,0.0,0.0,283.45,3543.0,Clouds
2018-09-30 20:00:00,90.0,0.0,0.0,282.76,2781.0,Clouds
2018-09-30 21:00:00,90.0,0.0,0.0,282.73,2159.0,Thunderstorm


In [ ]:


model_list = [
    'LastValueNaive'
]

model = AutoTS(
    forecast_length=49,
    frequency='infer',
    prediction_interval=0.95,
    max_generations=5,
    num_validations=2,
    validation_method='seasonal 168',
    model_list=model_list,
    transformer_list='all',
    models_to_validate=0.2,
    drop_most_recent=1,
    n_jobs='auto',
)

model = model.fit(
    df_wide,
    weights=weights_hourly,
)

prediction = model.predict()
forecasts_df = prediction.forecast
# prediction.long_form_results()

In [2]:
from autots import load_daily, model_forecast


df = load_daily(long=False)  # long or non-numeric data won't work with this function
df_forecast = model_forecast(
    model_name="AverageValueNaive",
    model_param_dict={'method': 'Mean'},
    model_transform_dict={
        'fillna': 'mean',
        'transformations': {'0': 'DifferencedTransformer'},
        'transformation_params': {'0': {}}
    },
    df_train=df,
    forecast_length=12,
    frequency='infer',
    prediction_interval=0.9,
    no_negatives=False,
    # future_regressor_train=future_regressor_train2d,
    # future_regressor_forecast=future_regressor_forecast2d,
    random_seed=321,
    verbose=0,
    n_jobs="auto",
)
df_forecast.forecast.head(10)

,wiki_United_States,wiki_Germany,wiki_List_of_highest-grossing_films,wiki_Jesus,wiki_Michael_Jackson,wiki_List_of_United_States_cities_by_population,wiki_Microsoft_Office,wiki_Google_Chrome,wiki_Periodic_table,wiki_Standard_deviation,...,wiki_Christmas,wiki_Chinese_New_Year,wiki_Thanksgiving,wiki_List_of_countries_that_have_gained_independence_from_the_United_Kingdom,wiki_History_of_the_hamburger,wiki_Elizabeth_II,wiki_William_Shakespeare,wiki_George_Washington,wiki_Cleopatra,wiki_all
2023-07-18,59360.935119,20129.885308,23347.583926,15510.601925,26382.291754,12791.257430,15285.013813,5537.549184,17820.21599,5355.010046,...,4399.446630,1476.892424,1959.848472,925.156969,1492.342821,25930.272499,12788.120134,57341.13018,192652.101298,7.766043e+08
2023-07-19,59307.870239,20133.770615,23347.167853,15512.203851,26379.583508,12792.514860,15291.027627,5536.098368,17825.43198,5357.020092,...,4395.893261,1467.784847,1959.696944,925.313939,1492.685643,25880.544998,12790.240268,57364.26036,192730.202595,7.766843e+08
2023-07-20,59254.805358,20137.655923,23346.751779,15513.805776,26376.875262,12793.772290,15297.041440,5534.647551,17830.64797,5359.030138,...,4392.339891,1458.677271,1959.545416,925.470908,1493.028464,25830.817497,12792.360402,57387.39054,192808.303893,7.767644e+08
2023-07-21,59201.740477,20141.541231,23346.335705,15515.407702,26374.167015,12795.029720,15303.055253,5533.196735,17835.86396,5361.040184,...,4388.786522,1449.569694,1959.393889,925.627878,1493.371285,25781.089996,12794.480536,57410.52072,192886.405190,7.768444e+08
2023-07-22,59148.675596,20145.426538,23345.919632,15517.009627,26371.458769,12796.287149,15309.069067,5531.745919,17841.07995,5363.050230,...,4385.233152,1440.462118,1959.242361,925.784847,1493.714106,25731.362495,12796.600670,57433.65090,192964.506488,7.769244e+08
2023-07-23,59095.610716,20149.311846,23345.503558,15518.611553,26368.750523,12797.544579,15315.082880,5530.295103,17846.29594,5365.060276,...,4381.679782,1431.354542,1959.090833,925.941817,1494.056928,25681.634994,12798.720804,57456.78108,193042.607786,7.770044e+08
2023-07-24,59042.545835,20153.197154,23345.087484,15520.213478,26366.042277,12798.802009,15321.096693,5528.844286,17851.51193,5367.070322,...,4378.126413,1422.246965,1958.939305,926.098786,1494.399749,25631.907493,12800.840938,57479.91126,193120.709083,7.770845e+08
2023-07-25,58989.480954,20157.082461,23344.671411,15521.815404,26363.334031,12800.059439,15327.110506,5527.393470,17856.72792,5369.080368,...,4374.573043,1413.139389,1958.787777,926.255756,1494.742570,25582.179992,12802.961072,57503.04144,193198.810381,7.771645e+08
2023-07-26,58936.416074,20160.967769,23344.255337,15523.417329,26360.625785,12801.316869,15333.124320,5525.942654,17861.94391,5371.090414,...,4371.019674,1404.031812,1958.636249,926.412725,1495.085391,25532.452491,12805.081206,57526.17162,193276.911679,7.772445e+08
2023-07-27,58883.351193,20164.853077,23343.839263,15525.019255,26357.917539,12802.574299,15339.138133,5524.491838,17867.15990,5373.100460,...,4367.466304,1394.924236,1958.484722,926.569694,1495.428213,25482.724990,12807.201339,57549.30180,193355.012976,7.773246e+08
